In [1]:
import matplotlib.pyplot as plt
import networkx
import numpy
from scipy.sparse import csr_matrix
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity


def load_csr_dataset(filename):
    ptr = [0]
    idx = []
    val = []
    with open(filename) as f:
        lines = f.readlines()
    for line in lines:
        line = line.split()
        length = len(line)
        for i in range(1, length, 2):
            idx.append(line[i-1])
            val.append(line[i])
        ptr.append(ptr[-1] + length / 2)
    return csr_matrix((val, idx, ptr), dtype=numpy.double)


def svd(data, new_dimension):
    transformer = TruncatedSVD(n_components=new_dimension)
    return transformer.fit_transform(data)


def dbscan(clusters, min_pts, eps):
    neighborhoods = []
    core = []
    border = []
    noise = []

    # Treat the centroid of each cluster as a point
    points = clusters.cluster_centers_
    cos_sims = cosine_similarity(points)

    # Find core points
    for i in range(len(points)):
        neighbors = []
        for p in range(len(points)):
            # If the distance is below eps, p is a neighbor
            if cos_sims[i][p] >= eps:
                neighbors.append(p)
        neighborhoods.append(neighbors)
        # If neighborhood has at least min_pts, i is a core point
        if len(neighbors) >= min_pts:
            core.append(i)

    print("core: ", core, len(core))

    # Find border points
    for i in range(len(points)):
        neighbors = neighborhoods[i]
        # Look at points that are not core points
        if len(neighbors) < min_pts:
            for j in range(len(neighbors)):
                # If one of its neighbors is a core, it is also in the core point's neighborhood,
                # thus it is a border point rather than a noise point
                if neighbors[j] in core:
                    border.append(i)
                    # Need at least one core point...
                    break

    print("border: ", border, len(border))

    # Find noise points
    for i in range(len(points)):
        if i not in core and i not in border:
            noise.append(i)

    print("noise", noise, len(noise))

    nodes = core + border
    graph = networkx.Graph()
    graph.add_nodes_from(nodes)

    # Create neighborhood
    for i in range(len(nodes)):
        for p in range(len(nodes)):
            # If the distance is below the threshold, add a link in the graph.
            if p != i and cos_sims[i][p] >= eps:
                graph.add_edges_from([(nodes[i], nodes[p])])

    return list(networkx.connected_components(graph))


def output_results(clusters, clusters_refined, filename):
    outliers_cluster = len(clusters_refined) + 1
    with open(filename, 'w') as f:
        for c in clusters.labels_:
            found = False
            for i, cr in enumerate(clusters_refined):
                if c in cr:
                    f.write(str(i+1) + '\n')
                    found = True
                    continue
            if not found:
                f.write(str(outliers_cluster) + '\n')

In [2]:
csr = load_csr_dataset('train_pr3.dat')
print("Shape: ", csr.shape)

Shape:  (8580, 126356)


In [3]:
#csr = svd(csr, 500)
clusters = MiniBatchKMeans(n_clusters=300).fit(csr)
print("KMeans clusters:", clusters.labels_, len(clusters.labels_))

KMeans clusters: [ 45 191 120 ... 191 291  29] 8580


In [11]:
clusters_refined = dbscan(clusters, 5, 0.35)
print("# clusters:", len(clusters_refined))
print("clusters: ", clusters_refined)
print(type(clusters_refined[0]))

core:  [2, 3, 4, 6, 8, 10, 12, 13, 14, 15, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 45, 47, 48, 49, 50, 53, 56, 57, 58, 59, 60, 63, 65, 69, 70, 72, 74, 75, 76, 77, 79, 81, 82, 84, 85, 86, 89, 90, 92, 95, 97, 98, 99, 100, 102, 104, 105, 109, 113, 114, 115, 116, 117, 118, 120, 121, 122, 125, 130, 133, 134, 136, 137, 138, 139, 142, 143, 145, 146, 147, 152, 153, 154, 155, 157, 160, 161, 162, 163, 164, 167, 168, 169, 170, 171, 172, 174, 178, 179, 180, 184, 185, 186, 187, 189, 190, 191, 192, 193, 194, 198, 199, 200, 202, 203, 205, 206, 207, 210, 211, 214, 216, 217, 218, 219, 221, 223, 225, 228, 230, 231, 233, 235, 238, 243, 245, 246, 248, 249, 255, 256, 257, 260, 262, 265, 266, 267, 269, 271, 272, 273, 275, 279, 280, 281, 283, 284, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 299] 185
border:  [5, 7, 11, 16, 55, 61, 62, 64, 78, 87, 88, 91, 101, 103, 106, 107, 112, 119, 123, 128, 132, 144, 149, 150, 156, 158, 173, 177, 

In [19]:
plt.subplot(111)
networkx.draw_circular(clusters_refined, with_labels=True, font_weight='bold')
plt.show()

['AmbiguousSolution', 'DiGraph', 'ExceededMaxIterations', 'Graph', 'GraphMLReader', 'GraphMLWriter', 'HasACycle', 'LCF_graph', 'MultiDiGraph', 'MultiGraph', 'NetworkXAlgorithmError', 'NetworkXError', 'NetworkXException', 'NetworkXNoCycle', 'NetworkXNoPath', 'NetworkXNotImplemented', 'NetworkXPointlessConcept', 'NetworkXTreewidthBoundExceeded', 'NetworkXUnbounded', 'NetworkXUnfeasible', 'NodeNotFound', 'NotATree', 'OrderedDiGraph', 'OrderedGraph', 'OrderedMultiDiGraph', 'OrderedMultiGraph', 'PlanarEmbedding', 'PowerIterationFailedConvergence', '__author__', '__bibtex__', '__builtins__', '__cached__', '__date__', '__doc__', '__file__', '__license__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'absolute_import', 'adamic_adar_index', 'add_cycle', 'add_path', 'add_star', 'adj_matrix', 'adjacency', 'adjacency_data', 'adjacency_graph', 'adjacency_matrix', 'adjacency_spectrum', 'adjlist', 'algebraic_connectivity', 'algebraicconnectivity', 'algorithms', 'all

ValueError: too many values to unpack (expected 2)